In [ ]:
# @title ## **1. RUN ME FIRST :)**
import os
import pathlib
import urllib.request
import shlex
import subprocess
import time
import contextlib
import platform

prockilled = False
newlines = ['\n', '\r\n']
currentsuboutput = ''
processid = None
everyprocessid = []
currentfoldertrack = []
currentcondition = ''

def unbuffered(proc, stream='stdout'):
    stream = getattr(proc, stream)
    with contextlib.closing(stream):
        while prockilled == False:
            out = []
            last = stream.read(1)
            # Don't loop forever
            if last == '' and proc.poll() is not None:
                break
            while last not in newlines:
                # Don't loop forever
                if last == '' and proc.poll() is not None:
                    break
                out.append(last)
                last = stream.read(1)
            out = ''.join(out)
            yield out

def transfare(todownload, folder, torename=''):
    todownload_s = shlex.quote(todownload)
    folder_s = shlex.quote(folder)
    if platform.system() == "Windows":
        localappdata = os.environ['LOCALAPPDATA']
        megagetloc = os.path.join(localappdata, "MEGAcmd", "mega-get.bat")
        global process
        process = run_with_subprocess(f"{shlex.quote(megagetloc)} {todownload_s} {folder_s}", folder, False, 'mega')
    else:
        cmd = ["mega-get", todownload_s, folder_s]
        proc = subprocess.Popen(
            cmd,
            stdout=subprocess.PIPE,
            stderr=subprocess.STDOUT,
            universal_newlines=True,
        )
        global processid
        global everyprocessid
        processid = proc.pid
        everyprocessid.append(processid)

        global currentsuboutput
        for line in unbuffered(proc):
            if prockilled == False:
                if not line.startswith("Download"):
                    currentsuboutput = line
                    print(f"\r{line}", end="")
                else:
                    print(f"\n{line}")
                    _ = subprocess.getoutput("pkill -f \"mega-cmd-server\"")
                    printdebug("MEGA server killed")

            else:
                currentsuboutput = ''
                print('\033[1;31mOperation Cancelled')
                print('\033[0m')
                global currentcondition
                currentcondition = 'Operation Cancelled'
                return
        currentsuboutput = ''
    if torename:
        listfilenew = os.listdir(folder)
        newerfoldertrack = []
        for file in listfilenew:
            pathoffile = os.path.join(folder, file)
            newerfoldertrack.append(pathoffile)
        checkrename = [x for x in newerfoldertrack if x not in currentfoldertrack]
        if checkrename:
            os.rename(checkrename[0], os.path.join(folder, torename))

def run_with_subprocess(cmd):
    process = subprocess.Popen(cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
    while True:
        output = process.stdout.readline()
        if output == b'' and process.poll() is not None:
            break
        if output:
            print(output.decode("utf-8").strip())
    return process.poll()

def installmega():
    HOME = os.path.expanduser("~")
    ocr_file = pathlib.Path(f"{HOME}/.ipython/ocr.py")
    if not ocr_file.exists():
        hCode = "https://raw.githubusercontent.com/biplobsd/" \
                    "OneClickRun/master/res/ocr.py"
        urllib.request.urlretrieve(hCode, str(ocr_file))

    from importlib.util import module_from_spec, spec_from_file_location
    ocr_spec = spec_from_file_location("ocr", str(ocr_file))
    ocr = module_from_spec(ocr_spec)
    ocr_spec.loader.exec_module(ocr)

    if not os.path.exists("/usr/bin/mega-cmd"):
        print('\033[1;32mInstalling MEGA ...')
        print('\033[0m')
        ocr.runSh('sudo apt-get -y update')
        ocr.runSh('sudo apt-get -y install libmms0 libc-ares2 libc6 libcrypto++6 libgcc1 libmediainfo0v5 libpcre3 libpcrecpp0v5 libssl1.1 libstdc++6 libzen0v5 zlib1g apt-transport-https')
        ocr.runSh('sudo curl -sL -o /var/cache/apt/archives/MEGAcmd.deb https://mega.nz/linux/MEGAsync/Debian_9.0/amd64/megacmd-Debian_9.0_amd64.deb', output=True)
        ocr.runSh('sudo dpkg -i /var/cache/apt/archives/MEGAcmd.deb', output=True)
        print('\033[1;32mMEGA is installed.')
        print('\033[0m')

def download_file(todownload, folder, torename=''):
    if not os.path.exists("/usr/bin/mega-cmd"):
        print("MEGA is not installed. Please run 'installmega()' first.")
        return
    transfare(todownload, folder, torename)

# Instalasi MEGA
installmega()

In [ ]:
# @title ## **2. MOUNT GOOGLE DRIVE**
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# @title ## **3. DOWNLOAD HERE**
# @markdown ##### Mega Link, example : https://mega.nz/file/gAxTWBAI#uL7EZay-OND5G6ELJlGfUNG0s7Q4TynZKqFdvs0v0tc
todownload = '' # @param {'type': 'string'}
# @markdown ##### Destination folder
folder = '' # @param {'type': 'string'}
torename = ''
download_file(todownload, folder, torename)